In [1]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm
import random
from sklearn.model_selection import GroupKFold,KFold,StratifiedKFold
from torch.utils.data import Dataset,DataLoader
import json
import re

In [2]:
SEED = 42
def seed_everything(seed):
    print(f'setting everything to seed {seed}')
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

setting everything to seed 42


In [3]:
train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
sample_submission = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/test'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [4]:
papers = {}
for paper_id in sample_submission['Id']:
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

In [5]:
## Literal Matching
## 训练集里面一共180个数据集标签
all_labels = set()

for label_1, label_2, label_3 in train[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
    all_labels.add(str(label_1).lower())
    all_labels.add(str(label_2).lower())
    all_labels.add(str(label_3).lower())
    
print(f'No. different labels: {len(all_labels)}')

No. different labels: 180


In [6]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()
def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

In [7]:
# 字符串匹配
literal_preds = []

for paper_id in tqdm(sample_submission['Id']):
    paper = papers[paper_id]
    text_1 = '. '.join(section['text'] for section in paper).lower()
    text_2 = totally_clean_text(text_1)
    
    labels = set()
    for label in all_labels:
        if label in text_1 or label in text_2:
            labels.add(clean_text(label))
    literal_preds.append('|'.join(labels))

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
class config:
    OVERLAP = 20
    MAX_LEN = 64
    MAX_LEN_INPUT = 128
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 16
    EPOCHS = 5
    BERT_PATH = "../input/bert-base-uncased/"
    ROBERTA_PATH = "../input/roberta-base"
    MODEL_PATH = "model.bin"
    TRAINING_FILE = "../input/coleridgeinitiative-show-us-the-data/train.csv"
    TOKENIZER = tokenizers.BertWordPieceTokenizer(
        f"{BERT_PATH}/vocab.txt", 
        lowercase=True
    )

In [9]:
def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union


connection_tokens = {'s', 'of', 'and', 'in', 'on', 'for', 'data', 'dataset'}
def find_mask_candidates(sentence):
    """
    Extract masking candidates for Masked Dataset Modeling from a given $sentence.
    A candidate should be a continuous sequence of at least 2 words, 
    each of these words either has the first letter in uppercase or is one of
    the connection words ($connection_tokens). Furthermore, the connection 
    tokens are not allowed to appear at the beginning and the end of the
    sequence.
    """
    def candidate_qualified(words):
        while len(words) and words[0].lower() in connection_tokens:
            words = words[1:]
        while len(words) and words[-1].lower() in connection_tokens:
            words = words[:-1]
        
        return len(words) >= 2
    
    candidates = []
    
    phrase_start, phrase_end = -1, -1
    for id in range(1, len(sentence)):
        word = sentence[id]
        if word[0].isupper() or word in connection_tokens:
            if phrase_start == -1:
                phrase_start = phrase_end = id
            else:
                phrase_end = id
        else:
            if phrase_start != -1:
                if candidate_qualified(sentence[phrase_start:phrase_end+1]):
                    candidates.append((phrase_start, phrase_end))
                phrase_start = phrase_end = -1
    
    if phrase_start != -1:
        if candidate_qualified(sentence[phrase_start:phrase_end+1]):
            candidates.append((phrase_start, phrase_end))
    
    return candidates

In [10]:
def clean_paper_sentence(s):
    """
    This function is essentially clean_text without lowercasing.
    """
    s = re.sub('[^A-Za-z0-9]+', ' ', str(s)).strip()
    s = re.sub(' +', ' ', s)
    return s

def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    MAX_LENGTH = config.MAX_LEN
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - config.OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

# 制作训练集
# Index(['Id', 'pub_title', 'dataset_title', 'dataset_label', 'cleaned_label'], dtype='object')

all_train_data = []
all_index = []
all_dataset = []
for paper_id in tqdm(train['Id'].unique()):
    # load paper
    temps = train[train['Id']==paper_id]
    
    paper = json.load(open(f'{paper_train_folder}/{paper_id}.json'))
    
    # extract sentences
    sentences = set([clean_paper_sentence(sentence) for section in paper 
                     for sentence in section['text'].split('.')
                    ])
    
    # 缩短句子
    sentences = shorten_sentences(sentences) # make sentences short
    
    # 筛选句子
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    # 选出带有data，study的句子
#     sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study']) or any(word in sentence.lower() for word in all_labels)]
    sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in all_labels)]

#     temps = []
#     for sentence in sentences:
#         for word in all_labels:
#             if word in sentence.lower():
#                 temps.append(sentence)
    for sentence in sentences:
        all_train_data.append(sentence)
        all_index.append(paper_id)
#     # 分词
#     sentences = [sentence.split() for sentence in sentences] # sentence = list of words
    
    
#     # mask
#     test_data = []
#     for sentence in sentences:
#         for phrase_start, phrase_end in find_mask_candidates(sentence):
#             dt_point = sentence[:phrase_start] + [mask] + sentence[phrase_end+1:]
#             test_data.append((' '.join(dt_point), ' '.join(sentence[phrase_start:phrase_end+1]))) # (masked text, phrase)
    
#     all_test_data.append(test_data)

  0%|          | 0/14316 [00:00<?, ?it/s]

In [11]:
train_data = pd.DataFrame(columns=['Id', 'sentence'])
train_data['Id'] = all_index
train_data['sentence'] = all_train_data

In [12]:
map_labels = {}
for word in all_labels:
    word = word.strip()
    map_labels[word] = []
    for wo in all_labels:
        wo = wo.strip()
        if wo!=word and wo in word:
            map_labels[word].append(wo)
#             already.add(wo)

# 检查出之前已经被添加过一次的
already = set()
for word in map_labels:
    word = word.strip()
    for wo in map_labels[word]:
        wo = wo.strip()
        if wo in already:
            map_labels[word].remove(wo)
        else:
            already.add(wo)

In [13]:
tempk = []
for k in map_labels:
    tempk += map_labels[k]
remove = []
for k in map_labels:
    if k in tempk:
        remove.append(k)

# 查看删除掉会不会漏掉
value_after_del = []

for k in map_labels:
    if k not in remove:
        value_after_del += map_labels[k]
value_after_del = set(value_after_del)
# 安全删除
for k in set(remove):
    # 查看
    for r in map_labels[k]:
        if r in value_after_del:
            map_labels[k].remove(r)
    if len(map_labels[k])==0:
        del map_labels[k]

In [14]:
def get_index(temp):  
    idxs0 = []
    idxs1 = []
    temp = temp.lower()
    already = set()
    for word in map_labels.keys():
        if word not in already:
            tempindex = temp.find(word)
            if tempindex!=-1:
                idxs0.append(tempindex)
                idxs1.append(tempindex+len(word)-1)
                already.add(word)
            else:
                # 找不到去找子集
                for wo in map_labels[word]:
                    if wo not in already:
                        tempindex = temp.find(wo)
                        if tempindex!=-1:
                            idxs0.append(tempindex)
                            idxs1.append(tempindex+len(wo)-1)
                            already.add(wo)
    #         for ind in (i for i, e in enumerate(temp) if e == word[0]):
    #             if temp[ind:ind+len_st] == word:
    #                 idx0 = ind
    #                 idx1 = ind + len_st - 1
    #                 idxs0.append(idx0)
    #                 idxs1.append(idx1)
    #                 break
    #             else:
    #                 if wo in map_labels[word]:
    #                     len_st = len(wo)
    #                     for ind in (i for i, e in enumerate(temp) if e == wo[0]):
    #                         if temp[ind:ind+len_st] == wo:
    #                             idx0 = ind
    #                             idx1 = ind + len_st - 1
    #                             idxs0.append(idx0)
    #                             idxs1.append(idx1)
    #                             break
    # 找出最长的
    mps = {}
    for k in range(len(idxs0)):
        if idxs0[k] not in mps:
            mps[idxs0[k]] = set()
        mps[idxs0[k]].add(idxs1[k])
    
    for k in mps:
        if len(mps[k])>=1:
            mps[k] = np.max(list(mps[k]))
    
    mps2 = {}
    for k in mps:
        s = mps[k]
        if s not in mps2:
            mps2[s] = set()
        mps2[s].add(k)

    for k in mps2:
        if len(mps2[k])>=1:
            mps2[k] = np.min(list(mps2[k]))
            
    
    idxs0 = []
    idxs1 = []
    for k in mps2:
        idxs1.append(k)
        idxs0.append(mps2[k])
    return {'idx0':idxs0,'idx1':idxs1}

In [15]:
train_data['len'] = train_data.sentence.map(lambda x: len(x.split(" ")))

In [16]:
train_data['ind'] = train_data.sentence.map(get_index)
train_data['data_count'] = train_data.ind.map(lambda x: len(x['idx0']))

In [17]:
train_data[train_data.data_count==2]

,Id,sentence,len,ind,data_count
25,e5a4c6af-0854-4f7e-b088-06f03c18a01a,The current study examined ethnic differences ...,63,"{'idx0': [158, 285], 'idx1': [191, 321]}",2
27,e5a4c6af-0854-4f7e-b088-06f03c18a01a,In the current study we examined the associati...,58,"{'idx0': [223, 292], 'idx1': [256, 328]}",2
62,a3c998b6-ea9d-4542-9267-68f718247ff5,7 College enrollment and completion data come ...,36,"{'idx0': [55, 129], 'idx1': [85, 165]}",2
77,f5cbbb74-6f6b-4f66-b86d-a6b22b8a1f45,re drawn from two large national longitudinal ...,21,"{'idx0': [109, 58], 'idx1': [139, 94]}",2
82,71ed23ed-a26d-4db3-9650-4e1be78c69a0,The research data used were generated by The T...,26,"{'idx0': [45, 113], 'idx1': [97, 149]}",2
...,...,...,...,...,...
52383,f5d1fdf9-9a7e-4f5f-8b28-c85377f43eb3,Nitrogen fertilizer applications are estimated...,36,"{'idx0': [147, 230], 'idx1': [167, 268]}",2
52553,e0788324-b55f-4f5b-92b6-2ee0b6b84722,The breeding bird survey data on the total spe...,44,"{'idx0': [187, 182], 'idx1': [225, 221]}",2
52704,57554069-793a-4e79-acb4-93aa63903b20,Geological Survey USGS North American Breeding...,38,"{'idx0': [23, 18], 'idx1': [61, 57]}",2
53766,64445ab3-6134-49fa-a2b5-87952c48d526,2020 Methods 233 SARS CoV 2 genome sequences w...,36,"{'idx0': [17, 69], 'idx1': [43, 98]}",2


In [18]:
def process_data(tweet, idx, tokenizer, max_len):
    # sentiment -> title
    tweet = tweet.lower()
#     idxs0,idx1 = None,None

    idxs0 = idx['idx0']
    idxs1 = idx['idx1']
    idlen = len(idxs0)
    
    sel = np.random.randint(0,idlen,1)[0]

    char_targets = [0] * len(tweet)
    mask_targets = [0] * len(tweet)
    for k in range(len(idxs0)):
        idx0 = idxs0[k]
        idx1 = idxs1[k]
        for ct in range(idx0, idx1 + 1):
            if k==sel:
                char_targets[ct] = 1
            else:
                mask_targets[ct] = 0
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids[1:-1]
    tweet_offsets = tok_tweet.offsets[1:-1]
    
    target_idx = []
    mask_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
        if sum(mask_targets[offset1: offset2]) > 0:
            mask_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

#     sentiment_id = {
#         'positive': 3893,
#         'negative': 4997,
#         'neutral': 8699
#     }
    
    input_ids = [101]  + input_ids_orig + [102]
#     token_type_ids = [0] + [1] * (len(input_ids_orig) + 1)
    # mask掉其他的
    mask = [1] * len(input_ids)
    for kk in mask_idx:
        mask[kk+1]= 0
    
    tweet_offsets = [(0, 0)] * 1 + tweet_offsets + [(0, 0)]
    targets_start += 1
    targets_end += 1

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
#         token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    # 处理分词后长度过长的问题
    if len(input_ids) > max_len:
        input_ids = input_ids[:max_len]
        mask = mask[:max_len]
        tweet_offsets = tweet_offsets[:max_len]
        if targets_end >= max_len:
            targets_end = max_len-1
        if targets_start >= max_len:
            targets_start = 0
            targets_end = 0
            
    return {
        'ids': input_ids,
        'mask': mask,
#         'token_type_ids': token_type_ids, 
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
#         'orig_selected': selected_text,
        'offsets': tweet_offsets
    }

In [19]:
class ColeridgeDataset:
    def __init__(self, tweet, idx):
        self.tweet = tweet
        self.idx = idx
#         self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN_INPUT
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.idx[item],
#             self.selected_text[item], 
            self.tokenizer,
            self.max_len
        )
        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
#             'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
#             'orig_selected': data["orig_selected"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [20]:
ss = None

In [21]:
class ColeridgeModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(ColeridgeModel, self).__init__(conf)
#         self.roberta = transformers.RobertaModel.from_pretrained(config.ROBERTA_PATH, config=conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids=None):
        out = self.bert(
            ids,
            attention_mask=mask,
#             token_type_ids=token_type_ids
        )
        out = out[2]
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [22]:
train_dataset = ColeridgeDataset(
    tweet=train_data.sentence.values,
    idx = train_data.ind
#     selected_text=df_train.selected_text.values
)

In [23]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [24]:
def compute_fbeta(y_true,
                  y_pred,
                  beta: float = 0.5) -> float:
    """Compute the Jaccard-based micro FBeta score.

    References
    ----------
    - https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation
    """

    def _jaccard_similarity(str1: str, str2: str) -> float:
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        if (len(a) + len(b) - len(c))<=0:
            return 0
        return float(len(c)) / (len(a) + len(b) - len(c))

    tp = 0  # true positive
    fp = 0  # false positive
    fn = 0  # false negative
    for ground_truth_list, predicted_string_list in zip(y_true, y_pred):
        predicted_string_list_sorted = sorted(predicted_string_list)
        for ground_truth in sorted(ground_truth_list):            
            if len(predicted_string_list_sorted) == 0:
                fn += 1
            else:
                similarity_scores = [
                    _jaccard_similarity(ground_truth, predicted_string)
                    for predicted_string in predicted_string_list_sorted
                ]
                matched_idx = np.argmax(similarity_scores)
                if similarity_scores[matched_idx] >= 0.5:
                    predicted_string_list_sorted.pop(matched_idx)
                    tp += 1
                else:
                    fn += 1
        fp += len(predicted_string_list_sorted)

    tp *= (1 + beta ** 2)
    fn *= beta ** 2
    fbeta_score = tp / (tp + fp + fn)
    return fbeta_score

In [25]:
from sklearn import metrics

def get_score(y_true, y_pred):
    scores = []
    acc = []
    for i in range(y_true.shape[1]):
        score = metrics.average_precision_score(y_true[:,i], y_pred[:,i])
        scores.append(score)
        yac = y_pred[:,i]>0.5
        acc.append(np.sum(yac==y_true[:,i])/y_true.shape[0])
    avg_score = np.mean(scores)
#     lb = y_true.argmax(axis=1)
#     prd = y_pred.argmax(axis=1)
#     acc = np.sum(lb==prd)/y_true.shape[0]
    scores.append(np.mean(acc))
    scores.extend(acc)
    return avg_score, scores

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
               
def HPA_auc(y_true, y_valid):
    return get_score(y_true,y_valid)
        
class HPAAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = None
        self.y_pred = None
        self.score = 0
        self.allscore = []
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.sigmoid().data.cpu().numpy()
        if self.y_true is None:
            self.y_true = y_true
            self.y_pred = y_pred
        else:
            self.y_true = np.concatenate((self.y_true, y_true))
            self.y_pred = np.concatenate((self.y_pred, y_pred))
        try:
            self.score,self.allscore = HPA_auc(self.y_true, self.y_pred)
        except:
            pass
    @property
    def avg(self):
        print(self.allscore[:19])
        print('acc---------------')
        print(self.allscore[20:])
        return self.score
    @property
    def alls(self):
        return self.allscore

In [26]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = AverageMeter()
    jaccards = AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
#         token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
        )
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        optimizer.step()
        scheduler.step()

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
#         jaccard_scores = []
        
        y_true = []
        y_pred = []
        for px, tweet in enumerate(orig_tweet):
            tempoff = offsets[px].numpy()
            idx_start=np.argmax(outputs_start[px, :])
            idx_end=np.argmax(outputs_end[px, :])
            yt = []
            yp = []
            
            for kk in range(idx_start,idx_end+1):
#                 print(tempoff[kk])
                yp.append(tweet[tempoff[kk][0]:tempoff[kk][1]])
            for kk in range(targets_start[px],targets_end[px]):
#                 print(tempoff[kk])
                yt.append(tweet[tempoff[kk][0]: tempoff[kk][1]])
            y_pred.append(' '.join(yp))
            y_true.append(' '.join(yt))
        # y_true
        # y_pred
#         for px, tweet in enumerate(orig_tweet):
#             selected_tweet = orig_selected[px]
#             tweet_sentiment = sentiment[px]
#             jaccard_score, _ = calculate_jaccard_score(
#                 original_tweet=tweet,
#                 target_string=selected_tweet,
#                 sentiment_val=tweet_sentiment,
#                 idx_start=np.argmax(outputs_start[px, :]),
#                 idx_end=np.argmax(outputs_end[px, :]),
#                 offsets=offsets[px]
#             )
#             jaccard_scores.append(jaccard_score)

        jaccards.update(compute_fbeta(y_true,y_pred), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg,jaccards=jaccards.avg)

In [27]:
def eval_fn(data_loader, model, device):
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            mask = d["mask"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"]

            ids = ids.to(device, dtype=torch.long)
    #         token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)
    
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
#                 token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
#             jaccard_scores = []
#             for px, tweet in enumerate(orig_tweet):
#                 selected_tweet = orig_selected[px]
#                 tweet_sentiment = sentiment[px]
#                 jaccard_score, _ = calculate_jaccard_score(
#                     original_tweet=tweet,
#                     target_string=selected_tweet,
#                     sentiment_val=tweet_sentiment,
#                     idx_start=np.argmax(outputs_start[px, :]),
#                     idx_end=np.argmax(outputs_end[px, :]),
#                     offsets=offsets[px]
#                 )
#                 jaccard_scores.append(jaccard_score)

            y_true = []
            y_pred = []
            for px, tweet in enumerate(orig_tweet):
                tempoff = offsets[px].numpy()
                idx_start=np.argmax(outputs_start[px, :])
                idx_end=np.argmax(outputs_end[px, :])
                yt = []
                yp = []

                for kk in range(idx_start,idx_end+1):
    #                 print(tempoff[kk])
                    yp.append(tweet[tempoff[kk][0]:tempoff[kk][1]])
                for kk in range(targets_start[px],targets_end[px]):
    #                 print(tempoff[kk])
                    yt.append(tweet[tempoff[kk][0]: tempoff[kk][1]])
                y_pred.append(' '.join(yp))
                y_true.append(' '.join(yt))

            jaccards.update(compute_fbeta(y_true,y_pred), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg,jaccards=jaccards.avg)
    
    print(f"Jaccard = {jaccards.avg}")
    return jaccards.avg

In [28]:
train_data['fold'] = 0
gf = GroupKFold(5)
for i,(train_ind,val_ind) in enumerate(gf.split(train_data.index,train_data.data_count,train_data.Id)):
    train_data.loc[val_ind,'fold'] = i

In [29]:
train_data.fold.value_counts()

3    10951
2    10951
1    10951
0    10951
4    10950
Name: fold, dtype: int64

In [30]:
def run(fold):
#     dfx = pd.read_csv(config.TRAINING_FILE)

    df_train = train_data[train_data.fold != fold].reset_index(drop=True)
    df_valid = train_data[train_data.fold == fold].reset_index(drop=True)
    
    train_dataset = ColeridgeDataset(
        tweet = df_train.sentence.values,
        idx = df_train.ind.values
    #     selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = ColeridgeDataset(
        tweet = df_valid.sentence.values,
        idx = df_valid.ind.values
    #     selected_text=df_train.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

    device = torch.device("cuda")
    model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
    model_config.output_hidden_states = True
    model = ColeridgeModel(conf=model_config)
    model.to(device)

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

#     es = utils.EarlyStopping(patience=2, mode="max")
    print(f"Training is Starting for fold={fold}")
    
    # I'm training only for 3 epochs even though I specified 5!!!
    for epoch in range(10):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        jaccard = eval_fn(valid_data_loader, model, device)
        
        print(f"Jaccard Score = {jaccard}")
        
        state = {'model':model.state_dict() }
        
        torch.save(state, f"model_{epoch}_{fold}_{jaccard}.bin")

#         if es.early_stop:
#             print("Early stopping")
#             break

In [31]:
run(0)

Training is Starting for fold=0


  0%|          | 0/685 [00:00<?, ?it/s]

  0%|          | 0/685 [00:00<?, ?it/s]

Jaccard = 0.5130762551262078
Jaccard Score = 0.5130762551262078


  0%|          | 0/685 [00:00<?, ?it/s]

...

  0%|          | 0/685 [00:00<?, ?it/s]

Jaccard = 0.6519251559696981
Jaccard Score = 0.6519251559696981


  0%|          | 0/685 [00:00<?, ?it/s]

  0%|          | 0/685 [00:00<?, ?it/s]

Jaccard = 0.6519251559696981
Jaccard Score = 0.6519251559696981
